# The Characterization of Single Amino Acid Polymorphism of UafA   
@author Changhua Yu  
@date 181009

## Overview 
- 180523: Literature Review  
- 180601: Pymol Visualization of the mutated sites  
    - The overall mapping of mutated sites
    - The overall mapping of mutated sites with site frequency as spectrum  
    - Mutated haplotypes visualization   
    - The potential functioning sites according literature  
    - The overlapping between mutated sites and potential functioning sites
- 180613: 
    1. Better visualization on superimposition between annotated regions and mutated sites
        - Generate the overlapping of all the mutated sites with each of the functional region        
        - Finish the labelling of func region in Pymol: As the Pymol display a label for each of the selected residues
          atoms, use a color scheme table instead
    2. Generate useful statistics from the sqlite3 outputs for the functional predictions   
        - Generate tables to answer: which site, AA change, site-AA pair and which region has the most frequent change
        - Output a correlation count matrix with a side length of N (N is the number of all distinct AA- site pairs)
        - Visualize spatial relationship of the mutated sites
    3. Structural prediction on N1 Region by online servers:
        - I-TASSER
        - Phyre2--homologous
        - Robbeta -- direct folded, only good for the shorter sequences ,N1  only e.g 
        - Pfam -- alignment based prediction
        - Research more on the prediction tool relating to single amino acid mutagenesis
- 180626:
    1. Better visualization on superimposition between annotated regions and mutated sites
        - Generate useful statistics from the sqlite3 outputs for the functional predictions   
        - Generate tables to answer: which site, AA change, site-AA pair and which region has the most frequent change 
        - Output a correlation count matrix with a side length of N (N is the number of all distinct AA- site pairs)
        Visualize spatial relationship of the mutated sites
- 180727:
    1. More Visualization
        - Generate the superimposition of the each haplotype with each annotated regions
        - Output a chart for assistive inspection 
    2. Re-output the tables
        - Fix the omission of deletion 
        - Fix the calculation of correlated occurence

The project is not done and this doc will continue as a documentation and implementation demo of the function from now on

## Fix the calculation of correlated occurence
- change FreqCorrMatrix.py
- the frequency should be (p(A1,A2))^2/p(A1)p(A2) from the mute_corr_count_2_180727 table
- insert into mute_corr_freq
- pattern_id| word_length| line| hap_id| corr_freq
- compare with the raw_count table

In [1]:
import sqlite3
import pandas as pd
# connect to sqlite3 local database

db_file = "../db/uafa.db"
conn = sqlite3.connect(db_file)
cur = conn.cursor()
cmd1 = '''select * from mute_corr_freq where corr_freq < 1.0 order by -1*corr_freq  limit 10;'''
df1 = pd.read_sql_query(cmd1, conn)
df1

,pattern_id,word_length,line,hap_id,corr_freq
0,5027,2,"40,37","45,57,26,59,29,63",0.857143
1,5047,2,"49,37","45,57,26,59,29,63",0.857143
2,5253,2,"90,98","2,3,6,7,8,9,10,11,12,13,14,15,16,19,22,23,24,2...",0.770833
3,4763,2,"72,65","57,59,45",0.750000
4,4793,2,"74,65","57,59,45",0.750000
5,5028,2,"40,32","45,57,26,59,29,63",0.750000
6,5029,2,"40,46","45,57,26,59,29,63",0.750000
7,5030,2,"40,51","45,57,26,59,29,63",0.750000
8,5048,2,"49,32","45,57,26,59,29,63",0.750000
9,5049,2,"49,46","45,57,26,59,29,63",0.750000


In [2]:
cmd2 = '''select * from mute_corr_count_2_180727 where count > 0 order by -1*count limit 10;'''
df2 = pd.read_sql_query(cmd2, conn)
df2

,pattern_id,word_length,line,hap_id,count
0,5253,2,"90,98","2,3,6,7,8,9,10,11,12,13,14,15,16,19,22,23,24,2...",37
1,5233,2,"1,3","35,37,39,40,42,45,46,47,48,51,53,54,55,56,57,5...",24
2,5234,2,"1,5","35,37,39,40,42,45,46,47,48,51,53,54,55,56,57,5...",24
3,5235,2,"1,100","35,37,39,40,42,45,46,47,48,51,53,54,55,56,57,5...",24
4,5236,2,"1,102","35,37,39,40,42,45,46,47,48,51,53,54,55,56,57,5...",24
5,5238,2,"1,98","35,37,39,40,42,45,46,47,48,51,53,54,55,56,57,5...",24
6,5239,2,"3,5","35,37,39,40,42,45,46,47,48,51,53,54,55,56,57,5...",24
7,5240,2,"3,100","35,37,39,40,42,45,46,47,48,51,53,54,55,56,57,5...",24
8,5241,2,"3,102","35,37,39,40,42,45,46,47,48,51,53,54,55,56,57,5...",24
9,5243,2,"3,98","35,37,39,40,42,45,46,47,48,51,53,54,55,56,57,5...",24


## Mutual Information Calculation of correlated text
- After learning the concept in BioE131, it seems that mutual information is what John-Marc initially suggests  
- A measure of how correlated the two objects are  
- I(x,y) = log2[P(x,y)/(P(x)*P(y))]

In [2]:
import sqlite3
import pandas as pd
# connect to sqlite3 local database

db_file = "../db/uafa.db"
conn = sqlite3.connect(db_file)
cur = conn.cursor()
cmd1 = '''select * from mute_corr_MI order by -1 * corr_MI  limit 10;'''
df1 = pd.read_sql_query(cmd1, conn)
df1

,patter_id,word_length,line,hap_id,corr_MI
0,107,2,"4,11",28,1.0
1,321,2,"7,41",40,1.0
2,406,2,"9,13",25,1.0
3,1164,2,"21,30",63,1.0
4,1166,2,"21,35",63,1.0
5,1175,2,"21,61",63,1.0
6,1178,2,"21,71",63,1.0
7,1179,2,"21,73",63,1.0
8,1180,2,"21,75",63,1.0
9,1181,2,"21,80",63,1.0


## Compare raw count, Bayesian and MI result on graph
- The in both chart and colored pymol model
